In [1]:
import numpy as np
import h5py
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

## Functions

In [2]:
def load_data(filename, dataname):
    with h5py.File(filename, 'r') as hf:
        x = hf[dataname][:]
    with h5py.File('EP_preprocessed_y.h5', 'r') as hf:
        y = hf['EP_preprocessed_y_dataset'][:]
    return x,y

In [3]:
def reshape_x(pre_x, channels, timepoints):
    new_x = np.reshape(pre_x, (pre_x.shape[0], channels, timepoints))
    return new_x

In [4]:
def train_test_split(file, data):
    x_,y_ = load_data(file, data)
    
    x_train, x_test = x_[:int(len(x_)*.80)], x_[-int(len(x_)*.20):]
    y_train, y_test = y_[:int(len(y_)*.80)], y_[-int(len(y_)*.20):]
    
    print("X train shape: ", x_train.shape)
    print("X test shape: ", x_test.shape)
    print("Y train shape: ", y_train.shape)
    print("Y test shape: ", y_test.shape)
    
    return x_train, x_test, y_train, y_test

In [5]:
def build_CNN(convLayer, denseLayer, nodes):
    model = keras.Sequential()

    model.add(keras.layers.Conv1D(nodes, 3, input_shape=(x_train.shape[1:]), data_format="channels_first", activation="relu"))
    model.add(keras.layers.MaxPooling1D(pool_size=2, padding='same'))

    for l in range(convLayer-1):
        model.add(keras.layers.Conv1D(nodes, 3, activation="relu"))
        model.add(keras.layers.MaxPooling1D(pool_size=2, padding='same'))

    model.add(keras.layers.Dropout(.5))
    model.add(keras.layers.Flatten())

    for l in range(denseLayer-1):
        model.add(keras.layers.Dense(nodes, activation="relu"))

    model.add(keras.layers.Dense(10, activation="softmax"))
    return model

In [6]:
def get_tensorboard(model_name, logdir):
    tensorboard = TensorBoard(log_dir=f'{logdir}\\{model_name}')
    #tensorboard = TensorBoard(log_dir=f'logs\\{model_name}')
    print("Model Name:", model_name)
    return tensorboard

## Hyper-Parameter Search

In [7]:
conv_layers = [3,2]
dense_layers = [2,1]
layer_sizes = [256,64,16]

In [8]:
# Data Files and Data Sets
data_files = ["EP_preprocessed_x.h5"]

data_sets = ["EP_preprocessed_x_dataset"]

In [9]:
for i in range(len(data_files)):
    x_train, x_test, y_train, y_test = train_test_split(data_files[i], data_sets[i])
    
    for conv_layer in conv_layers:
        for dense_layer in dense_layers:
            for layer_size in layer_sizes:

                # --- Tensorboard Callback ---
                NAME = f"CNN-{conv_layer}-conv-{dense_layer}-dense-{layer_size}-nodes-{int(time.time())}"
                tensorboard = get_tensorboard(model_name=NAME, logdir='logs_ConvNet')

                # --- Convolutional Neural Network ---
                model = build_CNN(conv_layer, dense_layer, layer_size)
                model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])    
                model.fit(x_train, y_train, epochs=10, validation_data=(x_test,y_test), callbacks=[tensorboard])

X train shape:  (51876, 14, 250)
X test shape:  (12969, 14, 250)
Y train shape:  (51876, 10)
Y test shape:  (12969, 10)
Model Name: CNN-3-conv-2-dense-256-nodes-1581191179
Train on 51876 samples, validate on 12969 samples
Epoch 1/10
51876/51876 [==============================] - 33s 627us/sample - loss: 2.3030 - accuracy: 0.1002 - val_loss: 2.3027 - val_accuracy: 0.1019
Epoch 2/10
51876/51876 [==============================] - 29s 565us/sample - loss: 2.3027 - accuracy: 0.1005 - val_loss: 2.3026 - val_accuracy: 0.1024
Epoch 3/10
51876/51876 [==============================] - 29s 557us/sample - loss: 2.3029 - accuracy: 0.1019 - val_loss: 2.3025 - val_accuracy: 0.1026
Epoch 4/10
51876/51876 [==============================] - 29s 557us/sample - loss: 2.3032 - accuracy: 0.0999 - val_loss: 2.3026 - val_accuracy: 0.0999
Epoch 5/10
51876/51876 [==============================] - 29s 556us/sample - loss: 2.3029 - accuracy: 0.0982 - val_loss: 2.3028 - val_accuracy: 0.1023
Epoch 6/10
51876/51876 

Model Name: CNN-3-conv-1-dense-16-nodes-1581192098
Train on 51876 samples, validate on 12969 samples
Epoch 1/10
51876/51876 [==============================] - 12s 228us/sample - loss: 2.3027 - accuracy: 0.0997 - val_loss: 2.3027 - val_accuracy: 0.0999
Epoch 2/10
51876/51876 [==============================] - 11s 203us/sample - loss: 2.3027 - accuracy: 0.1014 - val_loss: 2.3027 - val_accuracy: 0.0990
Epoch 3/10
51876/51876 [==============================] - 11s 203us/sample - loss: 2.3026 - accuracy: 0.1000 - val_loss: 2.3027 - val_accuracy: 0.0995
Epoch 4/10
51876/51876 [==============================] - 10s 199us/sample - loss: 2.3027 - accuracy: 0.0994 - val_loss: 2.3026 - val_accuracy: 0.1023
Epoch 5/10
51876/51876 [==============================] - 11s 205us/sample - loss: 2.3026 - accuracy: 0.1024 - val_loss: 2.3028 - val_accuracy: 0.0987
Epoch 6/10
51876/51876 [==============================] - 11s 204us/sample - loss: 2.3026 - accuracy: 0.0996 - val_loss: 2.3027 - val_accuracy: 

51876/51876 [==============================] - 22s 429us/sample - loss: 2.2968 - accuracy: 0.1050 - val_loss: 2.3058 - val_accuracy: 0.1018
Model Name: CNN-2-conv-1-dense-64-nodes-1581192909
Train on 51876 samples, validate on 12969 samples
Epoch 1/10
51876/51876 [==============================] - 12s 226us/sample - loss: 2.3030 - accuracy: 0.1025 - val_loss: 2.3027 - val_accuracy: 0.0999
Epoch 2/10
51876/51876 [==============================] - 11s 209us/sample - loss: 2.3029 - accuracy: 0.0991 - val_loss: 2.3028 - val_accuracy: 0.1004
Epoch 3/10
51876/51876 [==============================] - 11s 210us/sample - loss: 2.3028 - accuracy: 0.1003 - val_loss: 2.3026 - val_accuracy: 0.1021
Epoch 4/10
51876/51876 [==============================] - 11s 210us/sample - loss: 2.3025 - accuracy: 0.1017 - val_loss: 2.3028 - val_accuracy: 0.1030
Epoch 5/10
51876/51876 [==============================] - 11s 209us/sample - loss: 2.3024 - accuracy: 0.1000 - val_loss: 2.3031 - val_accuracy: 0.0975
Epoc